In [34]:
import os
import time
import pandas as pd
import pymysql
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm


def dev_db_conn(): 
    """개발 DB 접속 함수""" 
    conn = pymysql.connect( 
        host = 'db-7ma06.pub-cdb.ntruss.com', 
        user = 'redtable', 
        password = 'fpemxpdlqmf5491!@#', 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")
download_path = f"{file_path}사후면세점_이미지/"

#tour_api_key = "YYhySaaOw6OTs%2BYzUkMkE5kiPsS%2FvFxmIou76CqBxunnrFZvMffp7S6Tvmc6a0eZMegb2YyesOkOlqn%2FbTmS2Q%3D%3D" #Hob
tour_api_key = "nHrvc%2FKhbXDIYLMyX6f%2BYtGyufRKrQWaLS1TWkMK2tV6i%2BJH6uFcb7yDVAvbeOECYKlnughp%2BIfKNFDKo1Td9A%3D%3D" #Emile

In [22]:
free_tax_shopping = pd.read_excel(f"{file_path}사후면세점.xlsx", sheet_name="사후면세점_최종")
free_tax_shopping = free_tax_shopping.astype({"store_id":"int", "RSTR_ID":"int", "contentid":"int"})

In [23]:
free_tax_shopping

,store_id,RSTR_ID,contentid,매장명,지점명,지번주소,도로명주소,AREA1,AREA2,AREA3,위도,경도,우편번호,전화번호,영업날짜,영업시간,휴무일,매장라벨
0,1158146,300210189,2893628,더바디샵,김포롯데몰점,서울특별시 강서구 방화동 886,서울특별시 강서구 하늘길 38,서울특별시,강서구,방화동,37.571370,126.802960,7505.0,02-6116-5522,월~일요일,10:30~22:00,명절,"쇼핑,뷰티"
1,1158147,300210190,2893629,리노스,김해지점,경상남도 김해시 신문동 1414,경상남도 김해시 장유로 469,경상남도,김해시,신문동,35.185062,128.829596,51011.0,055-900-2361,월~일요일,11:00~21:00,명절,쇼핑
2,1158148,300210191,2893630,더바디샵,가로수길점,서울특별시 강남구 신사동 545-6,서울특별시 강남구 가로수길 46,서울특별시,강남구,신사동,37.521018,127.023018,6033.0,02-517-5285,월~일요일,11:00~22:00,NaN,"쇼핑,뷰티"
3,839833,422895,2893631,고디바,광화문점,서울특별시 종로구 세종로 202-1,서울특별시 종로구 세종대로 159,서울특별시,종로구,세종로,37.570605,126.976341,3183.0,070-736-7717,월~일요일,10:00~20:00,명절,쇼핑
4,1158149,300210192,2893632,더바디샵,메세나폴리스점,서울특별시 마포구 서교동 490,서울특별시 마포구 양화로 45,서울특별시,마포구,서교동,37.551501,126.913950,4036.0,NaN,월~일요일,11:00~22:00,명절,"쇼핑,뷰티"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7755,1165169,300217211,2932048,유니클로,롯데봉무점,대구광역시 동구 봉무동 1545,대구광역시 동구 팔공로49길 16,대구광역시,동구,봉무동,35.921548,128.637357,41026.0,NaN,월~일요일,11:00~21:00,명절,쇼핑
7756,1077404,300139897,2932049,이마트,천안점,충청남도 천안시 서북구 쌍용동 191,충청남도 천안시 서북구 충무로 187,충청남도,천안시 서북구,쌍용동,36.795925,127.127100,31180.0,041-620-1234,월~일요일,10:00~23:00,월2회 일요일,"쇼핑,뷰티"
7757,1165170,300217212,2932050,휠라키즈,신제주점,제주특별자치도 제주시 연동 273-5,제주특별자치도 제주시 신광로 51,제주특별자치도,제주시,연동,33.487483,126.490210,63123.0,064-757-5587,월~일요일,11:00~23:00,NaN,쇼핑
7758,1165171,300217213,2932120,모션,APM점,서울특별시 중구 을지로6가 18-35,서울특별시 중구 장충단로 253,서울특별시,중구,을지로6가,37.567487,127.008079,4564.0,NaN,월~일요일,10:30~16:30,명절,쇼핑


In [ ]:
data_list = []
except_list = []

for i in tqdm(range(len(free_tax_shopping["contentid"]))):
    try:
        content_id = free_tax_shopping["contentid"][i]
        time.sleep(10)
        url = f"http://apis.data.go.kr/B551011/KorService1/detailImage1?ServiceKey={tour_api_key}&contentId=2924134&MobileOS=ETC&MobileApp=AppTest&imageYN=Y&subImageYN=Y&numOfRows=10"
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")
        item = soup.find("item")
        if item is not None:
            data_list.append({
                "store_id": free_tax_shopping["store_id"][i],
                "contentid": item.contentid.get_text() if item.contentid else None,
                "originimgurl": item.originimgurl.get_text() if item.originimgurl else None,
                "imgname": item.imgname.get_text() if item.imgname else None,
                "smallimageurl": item.smallimageurl.get_text() if item.smallimageurl else None,
                "cpyrhtdivcd": item.cpyrhtdivcd.get_text() if item.cpyrhtdivcd else None,
                "serialnum": item.serialnum.get_text() if item.serialnum else None
            })

        else:
            except_list.append(content_id)
    except:
        except_list.append(content_id)
    
df = pd.DataFrame(data_list, columns=["store_id", "contentid", "originimgurl", "imgname", "smallimageurl", "cpyrhtdivcd", "serialnum"])